# dual regression
this notebook covers running melodic and dual regression in order to see if there are any differences between conditions (congruent and incongruent).

In [67]:
from nipype.interfaces.fsl.model import DualRegression, MELODIC, MultipleRegressDesign
from nipype import SelectFiles, Node, Workflow
import os
import re

In [37]:
# get inputs
templates = {'betaseries': 'sub-*/func/sub-*_trialtype-*congruent_zscore.nii.gz'}

# make a node
sf = Node(SelectFiles(templates),
          name='selectfiles')

# set to the top level directory where the data live
sf.inputs.base_directory = os.path.abspath('../')

# assign the output
betaseries = sf.run().outputs.betaseries

# get the mask:
mask = os.path.abspath('../group/group_bold_brainmask.nii.gz')

171227-22:45:48,784 workflow INFO:
	 Executing node selectfiles in dir: /tmp/tmpqi65hlpx/selectfiles
171227-22:45:48,787 workflow INFO:
	 Running node "selectfiles" ("nipype.interfaces.io.SelectFiles").


In [55]:
import nibabel as nib
arr = {}
# only use the nifti's where the participants were 100% correct
for fil in betaseries:
    arr.setdefault(nib.load(fil).shape[3], []).append(fil)
    
    

In [88]:
mask


'/media/Data/Documents/devel/nibetaseries_analysis/group/group_bold_brainmask.nii.gz'

In [63]:
melodic = Node(
            MELODIC(
                approach='tica',
                in_files=arr[24],
                dim=20,
                no_bet=True,
                mask=mask,
                out_dir=os.path.abspath('.cache/groupICA.out'),
                report=True),
            name='melodic')
melodic.run()

171227-23:36:26,757 workflow INFO:
	 Executing node melodic in dir: /tmp/tmp3283g8gi/melodic
171227-23:36:26,779 workflow INFO:
	 Running node "melodic" ("nipype.interfaces.fsl.model.MELODIC"), a CommandLine Interface with command:
melodic -i /media/Data/Documents/devel/nibetaseries_analysis/sub-01/func/sub-01_trialtype-congruent_zscore.nii.gz,/media/Data/Documents/devel/nibetaseries_analysis/sub-01/func/sub-01_trialtype-incongruent_zscore.nii.gz,/media/Data/Documents/devel/nibetaseries_analysis/sub-02/func/sub-02_trialtype-congruent_zscore.nii.gz,/media/Data/Documents/devel/nibetaseries_analysis/sub-02/func/sub-02_trialtype-incongruent_zscore.nii.gz,/media/Data/Documents/devel/nibetaseries_analysis/sub-03/func/sub-03_trialtype-congruent_zscore.nii.gz,/media/Data/Documents/devel/nibetaseries_analysis/sub-03/func/sub-03_trialtype-incongruent_zscore.nii.gz,/media/Data/Documents/devel/nibetaseries_analysis/sub-04/func/sub-04_trialtype-congruent_zscore.nii.gz,/media/Data/Documents/devel/ni

In [81]:
con_mean = ('overall_mean', 'T', ['congruent', 'incongruent'], [0.5, 0.5])
con_conmean = ('congruent_mean', 'T', ['congruent', 'incongruent'], [1.0, 0.0])
con_incmean = ('incongruent_mean', 'T', ['congruent', 'incongruent'], [0.0, 1.0])
con_congtinc = ('con>inc', 'T', ['congruent', 'incongruent'], [1.0, -1.0])
con_incgtcon = ('inc>con', 'T', ['congruent', 'incongruent'], [-1.0, 1.0])
contrasts = [con_mean, con_conmean, con_incmean, con_congtinc, con_incgtcon]


comp_con = re.compile('/media/Data/Documents/devel/nibetaseries_analysis/sub-[0-9]{2}/func/sub-[0-9]{2}_trialtype-congruent_zscore.nii.gz')
comp_inc = re.compile('/media/Data/Documents/devel/nibetaseries_analysis/sub-[0-9]{2}/func/sub-[0-9]{2}_trialtype-incongruent_zscore.nii.gz')

# congruent regressor
regressors = {'congruent': [], 'incongruent': []}
for fil in arr[24]:
    regressors['congruent'].append(int(bool(re.match(comp_con, fil))))
    regressors['incongruent'].append(int(bool(re.match(comp_inc, fil))))
# design = Node(MultipleRegressDesign())
# int(bool(re.match(comp_con, arr[24][1])))
model = MultipleRegressDesign(contrasts=contrasts, regressors=regressors)


In [59]:
os.getcwd()
os.chdir('/media/Data/Documents/devel/nibetaseries_analysis/code')

In [85]:
dual_regression = DualRegression(in_files=arr[24],
                                 group_IC_maps_4D='./.cache/groupICA.out/melodic_IC.nii.gz',
                                 n_perm=500,
                                 con_file='design.con',
                                 design_file='design.mat',
                                 out_dir='./.cache/dual_regression')
cmd = dual_regression.cmdline

'dual_regression ./.cache/groupICA.out/melodic_IC.nii.gz 1 design.mat design.con 500 ./.cache/dual_regression /media/Data/Documents/devel/nibetaseries_analysis/sub-01/func/sub-01_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-01/func/sub-01_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-02/func/sub-02_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-02/func/sub-02_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-03/func/sub-03_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-03/func/sub-03_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-04/func/sub-04_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-04/func/sub-04_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-05/f

In [91]:
%%bash
dual_regression /media/Data/Documents/devel/nibetaseries_analysis/code/.cache/groupICA.out/melodic_IC.nii.gz 1 design.mat design.con 500 /media/Data/Documents/devel/nibetaseries_analysis/group/group_bold_brainmask.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/code/.cache/DualRegression /media/Data/Documents/devel/nibetaseries_analysis/sub-01/func/sub-01_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-01/func/sub-01_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-02/func/sub-02_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-02/func/sub-02_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-03/func/sub-03_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-03/func/sub-03_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-04/func/sub-04_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-04/func/sub-04_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-05/func/sub-05_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-05/func/sub-05_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-06/func/sub-06_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-06/func/sub-06_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-07/func/sub-07_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-07/func/sub-07_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-08/func/sub-08_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-08/func/sub-08_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-09/func/sub-09_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-09/func/sub-09_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-10/func/sub-10_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-10/func/sub-10_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-11/func/sub-11_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-12/func/sub-12_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-12/func/sub-12_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-13/func/sub-13_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-13/func/sub-13_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-14/func/sub-14_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-14/func/sub-14_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-15/func/sub-15_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-15/func/sub-15_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-16/func/sub-16_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-16/func/sub-16_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-17/func/sub-17_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-17/func/sub-17_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-18/func/sub-18_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-18/func/sub-18_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-19/func/sub-19_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-19/func/sub-19_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-20/func/sub-20_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-20/func/sub-20_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-21/func/sub-21_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-22/func/sub-22_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-22/func/sub-22_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-24/func/sub-24_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-24/func/sub-24_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-25/func/sub-25_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-25/func/sub-25_trialtype-incongruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-26/func/sub-26_trialtype-congruent_zscore.nii.gz /media/Data/Documents/devel/nibetaseries_analysis/sub-26/func/sub-26_trialtype-incongruent_zscore.nii.gz

doing the dual regressions
sorting maps and running randomise


/usr/share/fsl/5.0/bin/fsl_sub: 410: exit: Illegal number: -1
/usr/share/fsl/5.0/bin/fsl_sub: 410: exit: Illegal number: -1


had to edit dual_regression to this so I could add my own mask

```bash
#!/bin/sh

#   dual_regression - take group-ICA maps (etc) and get subject-specific versions of them (and associated timecourses)
#
#   Stephen Smith and Christian Beckmann, FMRIB Image Analysis Group
#
#   Copyright (C) 2011-2012 University of Oxford
#
#   Part of FSL - FMRIB's Software Library
#   http://www.fmrib.ox.ac.uk/fsl
#   fsl@fmrib.ox.ac.uk
#
#   Developed at FMRIB (Oxford Centre for Functional Magnetic Resonance
#   Imaging of the Brain), Department of Clinical Neurology, Oxford
#   University, Oxford, UK
#
#
#   LICENCE
#
#   FMRIB Software Library, Release 5.0 (c) 2012, The University of
#   Oxford (the "Software")
#
#   The Software remains the property of the University of Oxford ("the
#   University").
#
#   The Software is distributed "AS IS" under this Licence solely for
#   non-commercial use in the hope that it will be useful, but in order
#   that the University as a charitable foundation protects its assets for
#   the benefit of its educational and research purposes, the University
#   makes clear that no condition is made or to be implied, nor is any
#   warranty given or to be implied, as to the accuracy of the Software,
#   or that it will be suitable for any particular purpose or for use
#   under any specific conditions. Furthermore, the University disclaims
#   all responsibility for the use which is made of the Software. It
#   further disclaims any liability for the outcomes arising from using
#   the Software.
#
#   The Licensee agrees to indemnify the University and hold the
#   University harmless from and against any and all claims, damages and
#   liabilities asserted by third parties (including claims for
#   negligence) which arise directly or indirectly from the use of the
#   Software or the sale of any products based on the Software.
#
#   No part of the Software may be reproduced, modified, transmitted or
#   transferred in any form or by any means, electronic or mechanical,
#   without the express permission of the University. The permission of
#   the University is not required if the said reproduction, modification,
#   transmission or transference is done without financial return, the
#   conditions of this Licence are imposed upon the receiver of the
#   product, and all original and amended source code is included in any
#   transmitted product. You may be held legally responsible for any
#   copyright infringement that is caused or encouraged by your failure to
#   abide by these terms and conditions.
#
#   You are not permitted under this Licence to use this Software
#   commercially. Use for which any financial return is received shall be
#   defined as commercial use, and includes (1) integration of all or part
#   of the source code or the Software into a product for sale or license
#   by or on behalf of Licensee to third parties or (2) use of the
#   Software or any derivative of it for research with the final aim of
#   developing software products for sale or license to a third party or
#   (3) use of the Software or any derivative of it for research with the
#   final aim of developing non-software products for sale or license to a
#   third party, or (4) use of the Software to provide any service to an
#   external organisation for which payment is received. If you are
#   interested in using the Software commercially, please contact Isis
#   Innovation Limited ("Isis"), the technology transfer company of the
#   University, to negotiate a licence. Contact details are:
#   innovation@isis.ox.ac.uk quoting reference DE/9564.
export LC_ALL=C


Usage() {
    cat <<EOF

dual_regression v0.5 (beta)

***NOTE*** ORDER OF COMMAND-LINE ARGUMENTS IS DIFFERENT FROM PREVIOUS VERSION

Usage: dual_regression <group_IC_maps> <des_norm> <design.mat> <design.con> <n_perm> <output_directory> <input1> <input2> <input3> .........
e.g.   dual_regression groupICA.gica/groupmelodic.ica/melodic_IC 1 design.mat design.con 500 grot \`cat groupICA.gica/.filelist\`

<group_IC_maps_4D>            4D image containing spatial IC maps (melodic_IC) from the whole-group ICA analysis
<des_norm>                    0 or 1 (1 is recommended). Whether to variance-normalise the timecourses used as the stage-2 regressors
<design.mat>                  Design matrix for final cross-subject modelling with randomise
<design.con>                  Design contrasts for final cross-subject modelling with randomise
<n_perm>                      Number of permutations for randomise; set to 1 for just raw tstat output, set to 0 to not run randomise at all.
<output_directory>            This directory will be created to hold all output and logfiles
<input1> <input2> ...         List all subjects' preprocessed, standard-space 4D datasets

<design.mat> <design.con>     can be replaced with just
-1                            for group-mean (one-group t-test) modelling.
If you need to add other randomise options then edit the line after "EDIT HERE" in the dual_regression script

EOF
    exit 1
}

############################################################################

[ "$6" = "" ] && Usage

ORIG_COMMAND=$*

ICA_MAPS=`${FSLDIR}/bin/remove_ext $1` ; shift

DES_NORM=--des_norm
if [ $1 = 0 ] ; then
  DES_NORM=""
fi ; shift

if [ $1 = "-1" ] ; then
  DESIGN="-1"
  shift
else
  dm=$1
  dc=$2
  DESIGN="-d $1 -t $2"
  shift 2
fi

NPERM=$1 ; shift

MASK=$1 ; shift

OUTPUT=`${FSLDIR}/bin/remove_ext $1` ; shift

while [ _$1 != _ ] ; do
  INPUTS="$INPUTS `${FSLDIR}/bin/remove_ext $1`"
  shift
done

############################################################################

mkdir $OUTPUT
LOGDIR=${OUTPUT}/scripts+logs
mkdir $LOGDIR
echo $ORIG_COMMAND > $LOGDIR/command
if [ "$DESIGN" != -1 ] ; then
  /bin/cp $dm $OUTPUT/design.mat
  /bin/cp $dc $OUTPUT/design.con
fi

# echo "creating common mask"
# j=0
# for i in $INPUTS ; do
#   echo "$FSLDIR/bin/fslmaths $i -Tstd -bin ${OUTPUT}/mask_`${FSLDIR}/bin/zeropad $j 5` -odt char" >> ${LOGDIR}/drA
#   j=`echo "$j 1 + p" | dc -`
# done
# ID_drA=`$FSLDIR/bin/fsl_sub -T 10 -N drA -l $LOGDIR -t ${LOGDIR}/drA`
# cat <<EOF > ${LOGDIR}/drB
# #!/bin/sh
# \$FSLDIR/bin/fslmerge -t ${OUTPUT}/maskALL \`\$FSLDIR/bin/imglob ${OUTPUT}/mask_*\`
# \$FSLDIR/bin/fslmaths $OUTPUT/maskALL -Tmin $OUTPUT/mask
# \$FSLDIR/bin/imrm $OUTPUT/mask_*
# EOF
# chmod a+x ${LOGDIR}/drB
# ID_drB=`$FSLDIR/bin/fsl_sub -j $ID_drA -T 5 -N drB -l $LOGDIR ${LOGDIR}/drB`

echo "doing the dual regressions"
j=0
for i in $INPUTS ; do
  s=subject`${FSLDIR}/bin/zeropad $j 5`
  echo "$FSLDIR/bin/fsl_glm -i $i -d $ICA_MAPS -o $OUTPUT/dr_stage1_${s}.txt --demean -m $MASK ; \
        $FSLDIR/bin/fsl_glm -i $i -d $OUTPUT/dr_stage1_${s}.txt -o $OUTPUT/dr_stage2_$s --out_z=$OUTPUT/dr_stage2_${s}_Z --demean -m ${MASK} $DES_NORM ; \
        $FSLDIR/bin/fslsplit $OUTPUT/dr_stage2_$s $OUTPUT/dr_stage2_${s}_ic" >> ${LOGDIR}/drC
  j=`echo "$j 1 + p" | dc -`
done
ID_drC=`$FSLDIR/bin/fsl_sub -j $ID_drB -T 30 -N drC -l $LOGDIR -t ${LOGDIR}/drC`

echo "sorting maps and running randomise"
j=0
Nics=`$FSLDIR/bin/fslnvols $ICA_MAPS`
while [ $j -lt $Nics ] ; do
  jj=`$FSLDIR/bin/zeropad $j 4`

  RAND=""
  if [ $NPERM -eq 1 ] ; then
    RAND="$FSLDIR/bin/randomise -i $OUTPUT/dr_stage2_ic$jj -o $OUTPUT/dr_stage3_ic$jj -m $MASK $DESIGN -n 1 -V -R"
  fi
  if [ $NPERM -gt 1 ] ; then
    # EDIT HERE
    RAND="$FSLDIR/bin/randomise -i $OUTPUT/dr_stage2_ic$jj -o $OUTPUT/dr_stage3_ic$jj -m $MASK $DESIGN -n $NPERM -T -V"
  fi

  echo "$FSLDIR/bin/fslmerge -t $OUTPUT/dr_stage2_ic$jj \`\$FSLDIR/bin/imglob $OUTPUT/dr_stage2_subject*_ic${jj}.*\` ; \
        $FSLDIR/bin/imrm \`\$FSLDIR/bin/imglob $OUTPUT/dr_stage2_subject*_ic${jj}.*\` ; $RAND" >> ${LOGDIR}/drD
  j=`echo "$j 1 + p" | dc -`
done
ID_drD=`$FSLDIR/bin/fsl_sub -j $ID_drC -T 60 -N drD -l $LOGDIR -t ${LOGDIR}/drD`
```
for it to almost work (had to add python2 to imglob since i use python3 by default



## results

ic0010 is the fronto-parietal network, this is only a little significant in a couple voxels, but it's in support of the hypothesis :

"differences in between the congruent and incongruent conditions should be seen in the fronto-parietal network using betaseries"

In this instance the direction of the result is greater "network activation?" of the incongruent condition over the congruent condition. None of the other components (19) in the dual regression model reached significance with randomise, this provide tentative evidence of the betaseries method being useful in the context of differentiating conditions in a cognitive control task. Specifically, betaseries may capture the differences between network activation in relevant (the manipulated) networks in the task.
dr_stage3_ic0010_tfce_corrp_tstat5.nii.gz
0.000000 0.988000 (the significant voxels were near the left? parietal region)